# Index Fund Arbitrage

## Introduction

An index fund will hold a basket of assets for some sort of general investment thesis. This allows individuals to buy and sell the index fund shares but essentially "own" the basket instead of needing to form the basket themselves.

Every day, an index fund's NAV, Net Asset Value, is calculated and it represents the underlying value of the basket. The actual share price for the index fund shares can be at a premium (higher than the NAV) or at a discount (lower than the NAV). However, we expect an equilibrium over time where there is on average no premium or discount.

This example will track the price of a basket of goods as well as an index fund and try to see what dynamics might arise in terms of potential arbitrage. We will assume there are two types of actors:

1. Arbitrage Actor: This actor brings the price of the index fund in line with the basket of goods through arbitrage.
2. Momentum Trader: This actor will bet on momentum.

## Historical Set Up

Historically, we will say that the basket of goods and the index fund both gain and lose an amount which is highly correlated with one and other. There are also actions which are taken on the index fund. The steps for creating this series of hypothetical data is below:

1. Basket of goods grows in percentage by $r_b = N(\mu_1, \sigma^2_1)$
2. The index fund grows in percentage by $r_i = \lambda r_b + (1-\lambda)\cdot N(\mu_2, \sigma^2_2)$
3. The arbitrage trader brings the index fund in line with the basket of goods if $|P_b - P_i| > \theta_1$.
4. The momentum trader will trade with the momentum if the m day return of the index - $\mu_1*m$ on an absolute basis is greater than $\theta_2$ growing or shrinking the index fund value by  .5% for market impact.

We will make a very heroic assumption that we can parse out pure price movements and the price of the stock after including the actor actions.

In [1]:
import sqlite3
import pandas as pd
import numpy as np
con = sqlite3.connect('arb.db')
np.random.seed(0)

mu1 = .01
std1 = .05
lambda1 = .9
mu2 = .015
std2 = .1
n = 100
r_b = np.random.normal(mu1, std1, n)
r_i = lambda1 * r_b + (1 - lambda1) * np.random.normal(mu2, std2, n)

t = 0
P_b = 100
P_i = 100
P_b_historical = [100]
P_i_historical = [100]

theta1 = 10
arbitrage_trades = []

lookback = 5
theta2 = .1
momentum_buy = []
momentum_sell = []

for _ in range(n):
    P_b = P_b * (1 + r_b[t])
    P_i = P_i * (1 + r_i[t])
    
    #Check arbitrage trade
    if abs(P_b - P_i) > theta1:
        arbitrage_trades.append(t)
        P_i = P_b
    
    #Check momentum trade
    lookback_i = max(t - lookback + 1, 0)
    period_return = P_i / P_i_historical[lookback_i] - 1 - mu1 * lookback
    if abs(period_return) > theta2:
        if period_return > 0:
            momentum_buy.append(t)
            P_i = P_i * 1.025
        else:
            momentum_sell.append(t)
            P_i = P_i * (1-.025)
    
    P_b_historical.append(P_b)
    P_i_historical.append(P_i)
    
    t += 1
    
pure_returns = pd.DataFrame(zip(r_i, r_b))
pure_returns.columns = ["index_return", "basket_return"]
pure_returns["t"] = pure_returns.index

prices = pd.DataFrame(zip(P_i_historical, P_b_historical))
prices.columns = ["index_price", "basket_price"]
prices["t"] = prices.index


trades = [[x, "Arbitrage"] for x in arbitrage_trades] + [[x, "Momentum Buy"] for x in momentum_buy] + [[x, "Momentum Sell"] for x in momentum_sell]
trades = pd.DataFrame(trades)
trades.columns = ["time", "trade"]

pure_returns.to_sql("pure_returns", con, index=False, if_exists='replace')
prices.to_sql("prices", con, index=False, if_exists='replace')
trades.to_sql("trades", con, index=False, if_exists='replace')